# 04-notebook-rough-draft for final project

I am working on the Kaggle Grupo Bimbo competition dataset for this project. 
Link to Grupo Bimbo Kaggle competition: [Kaggle-GrupoBimbo](https://www.kaggle.com/c/grupo-bimbo-inventory-demand)

In [1]:
import numpy as np
import pandas as pd
from sklearn import cross_validation
from sklearn import metrics
from sklearn import linear_model
from sklearn import ensemble
#QUESTION - what is diff bw random forest classifier and rf regressor?

#import seaborn as sns
import matplotlib.pyplot as plt
#sns.set(style="whitegrid", font_scale=1)
%matplotlib inline

![](../assets/images/workflow/data-science-workflow-01.png)

## Part 1. Identify the Problem



**Problem**: Given various sales/client/product data, we want to predict demand for each product at each store on a weekly basis. Per the train dataset, the average demand for a product at a store per week is 7.2 units. However, this does not factor in cases in which store managers under-predict demand for a product which we can see when returns=0 for that week. There are 74,180,464 records in the train data, of which 71,636,003 records have returns=0 or approx 96%. This generally means that managers probably often under predict product demand (unless that are exact on the money, which seems unlikely). 

**Goals**: The goal is to predict demand for each product at each store on a weekly basis while avoiding under-predicting demand.

**Hypothesis**: As stated previously, the average product demand at a store per week is 7.2 units per the train data. However, given the likelihood of managers underpredicint product demand, I hypothesize a good model should return a number higher than 7.2 units to more accurately predict demand.

![](../assets/images/workflow/data-science-workflow-02.png)

## Part 2. Acquire the Data

Kaggle has provided five files for this dataset:  
_train.csv_: Use for building a model (contains target variable "Demanda_uni_equil")  
_test.csv_: Use for submission file (fill in for target variable "Demanda_uni_equil")
_cliente_tabla.csv_: Contains client names (can be joined with train/test on Cliente_ID)
_producto_tabla.csv_: Contains product names (can be join with train/test on Producto_ID)
_town_state.csv_: Contains town and state (can be join with train/test on Agencia_ID)


**Notes**: I will further split _train.csv_ to generate my own cross validation set. However, I will use all of _train.csv_ to train my final model since Kaggle has already supplied a test dataset. Additionally, I am only using a random 10% of the train data given to me for EDA and model development. Using the entire train dataset proved to be too time consuming for the quick iternations needed for initial modeling building and EDA efforts. I plan to use 100% of the train dataset once I build a model I'm comfortable with. I may have to explore using EC2 for this effort.

In [2]:
# Load train data
# Given size of training data, I chose to use only 10% for speed reasons
# QUESTION - how can i randomize with python? i used sql to create the random sample below.
df_train = pd.read_csv("../train_random10percent.csv")

# Check head
df_train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1112,1,1210,4619860,1150,2,27.92,0,0.0,2
1,3,1112,1,1210,4619860,1160,1,18.86,0,0.0,1
2,3,1112,1,1210,4619860,1182,4,55.76,0,0.0,4
3,3,1112,1,1210,4683304,1109,2,30.02,0,0.0,2
4,3,1112,1,1210,4683304,1125,7,67.20,0,0.0,7


In [3]:
# Load test data
df_test = pd.read_csv("../test.csv")

# Check head. I noticed that I will have to drop certain columns so that test and train sets have the same features. 
df_test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [4]:
#given that i cannot use a significant amount of variables in train data, i created additoinal features using the mean
#i grouped on product id since i will ultimately be predicting demand for each product

df_train_mean = df_train.groupby('Producto_ID').mean().add_suffix('_mean').reset_index()
df_train_mean.head()

,Producto_ID,Semana_mean,Agencia_ID_mean,Canal_ID_mean,Ruta_SAK_mean,Cliente_ID_mean,Venta_uni_hoy_mean,Venta_hoy_mean,Dev_uni_proxima_mean,Dev_proxima_mean,Demanda_uni_equil_mean
0,41,6.500000,2257.714286,7.000000,3476.428571,9.927571e+05,261.857143,4733.798571,6.428571,116.550000,261.857143
1,53,7.000000,1868.818182,4.000000,7614.636364,2.606382e+06,219.909091,3144.700000,0.000000,0.000000,219.909091
2,72,6.139906,2569.569063,1.085722,2370.468701,1.505971e+06,4.918930,18.179211,0.083738,0.307245,4.869604
3,73,6.032710,2207.965045,1.183846,1291.151503,1.302741e+06,3.213304,68.875936,0.077698,1.659034,3.140966
4,100,5.928571,1827.500000,7.428571,3385.714286,2.014600e+06,10.642857,190.905714,9.642857,209.871429,10.642857


In [5]:
#from above, adding 2 additional features, the average sales units and the average demand
df_train2 = df_train.merge(df_train_mean[['Producto_ID','Venta_uni_hoy_mean', 'Demanda_uni_equil_mean']],how='inner',on='Producto_ID')
df_train2.sample(5)

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil,Venta_uni_hoy_mean,Demanda_uni_equil_mean
5478775,7,1952,1,2012,617699,43065,8,54.08,0,0.0,8,2.728514,2.674953
2571944,8,1534,1,1201,623664,41938,9,89.19,0,0.0,9,3.326363,3.266425
1137377,3,1333,1,1228,706050,1240,1,8.38,0,0.0,1,5.700916,5.633376
6246221,3,25759,1,2129,4585378,31470,2,21.52,0,0.0,2,5.084246,5.055041
6952006,3,4049,1,1272,1986127,34213,2,39.88,0,0.0,2,5.936357,5.884776


In [6]:
# Adding features to the test set in order to match train set
df_test2 = df_test.merge(df_train_mean[['Producto_ID','Venta_uni_hoy_mean', 'Demanda_uni_equil_mean']],how='left',on='Producto_ID')
df_test2.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy_mean,Demanda_uni_equil_mean
0,0,11,4037,1,2209,4639078,35305,9.171683,9.132431
1,1,11,2237,1,1226,4705135,1238,3.217857,3.161574
2,2,10,2045,1,2831,4549769,32940,4.042903,4.018765
3,3,11,1227,1,4448,4717855,43066,2.499306,2.439875
4,4,11,1219,1,1130,966351,1277,1.500000,1.500000


![](../assets/images/workflow/data-science-workflow-03-05.png)

## Part 3. Parse, Mine, and Refine the data

Perform exploratory data analysis and verify the quality of the data.

### Check columns and counts to drop any non-generic or near-empty columns

In [ ]:
# Check columns
print "train dataset columns:"
print df_train2.columns.values
print 
print "test dataset columns:"
print df_test2.columns.values

In [ ]:
# Check counts
print "train dataset counts:"
print df_train2.count()
print
print "test dataset counts:"
print df_test2.count()

### Check for missing values and drop or impute

In [ ]:
# Check counts for missing values in each column
print "train dataset missing values:"
print df_train2.isnull().sum()
print
print "test dataset missing values:"
print df_test2.isnull().sum()

### Wrangle the data to address any issues from above checks

In [7]:
# Drop columns not included in test dataset
df_train2 = df_train2.drop(['Venta_uni_hoy', 'Venta_hoy', 'Dev_uni_proxima', 'Dev_proxima'], axis=1)

# Check data
df_train2.head()


,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Demanda_uni_equil,Venta_uni_hoy_mean,Demanda_uni_equil_mean
0,3,1112,1,1210,4619860,1150,2,6.242652,6.191044
1,3,1111,1,1011,1723873,1150,1,6.242652,6.191044
2,3,1111,1,1011,4314785,1150,1,6.242652,6.191044
3,3,1111,1,1012,66252,1150,3,6.242652,6.191044
4,3,1111,1,1012,210290,1150,17,6.242652,6.191044


In [8]:
# Drop blank values in test set and replace with mean

# Replace missing values for venta_uni_hoy_mean using mean
df_test2.loc[(df_test2['Venta_uni_hoy_mean'].isnull()), 'Venta_uni_hoy_mean'] = df_test2['Venta_uni_hoy_mean'].dropna().mean()

# Replace missing values for demand using mean
df_test2.loc[(df_test2['Demanda_uni_equil_mean'].isnull()), 'Demanda_uni_equil_mean'] = df_test2['Demanda_uni_equil_mean'].dropna().mean()

print "test dataset missing values:"
print df_test2.isnull().sum()

test dataset missing values:
id                        0
Semana                    0
Agencia_ID                0
Canal_ID                  0
Ruta_SAK                  0
Cliente_ID                0
Producto_ID               0
Venta_uni_hoy_mean        0
Demanda_uni_equil_mean    0
dtype: int64


### Perform exploratory data analysis

In [ ]:
# Get summary statistics for data
df_train2.describe()

In [ ]:
#Show box plot of demand by week
sns.factorplot(
    x='Semana',
    y='Demanda_uni_equil',
    data=df_train2,
    kind='box')

### Check and convert all data types to numerical

In [ ]:
# Check data types
df_train.dtypes

#these are all numerical but are not continuous values and therefore don't have relative significant to one another, except for week
#however, creating dummy variables for all these is too memory intensive. as such, might have to explore using a random forest model
#in addition to the linear regression model

![](../assets/images/workflow/data-science-workflow-06.png)

## Part 4. Build a Model

Create a cross validation split, select and build a model, evaluate the model, and refine the model

### Create cross validation sets

In [ ]:
#create cross validation sets

#set target variable name
target = 'Demanda_uni_equil'

#set X and y
X = df_train2.drop([target], axis=1)
y = df_train2[target]

# create separate training and test sets with 60/40 train/test split
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size= .4) 

#QUESTION - do i have to cross validate when using kaggle data? prob not.

### Build a model

In [ ]:
#create linear regression object
#lm = linear_model.LinearRegression()

#create random forest object
#rf = ensemble.RandomForestClassifier(n_estimators=10) - did not work due to memory errors during fitting
rf = ensemble.RandomForestRegressor(n_estimators=100)

In [ ]:
#train the model using the training data
rf.fit(X_train, y_train)

### Evaluate the model

In [ ]:
# Check score on test set
print "Score: %0.3f" % rf.score(X_test,y_test)
#score on test set using 100 n_estimators was .46

![](../assets/images/workflow/data-science-workflow-07.png)

## Part 5: Present the Results

Generate summary of findings and kaggle submission file.

NOTE: For the purposes of generating summary narratives and kaggle submission, we can train the model on the entire training data provided in _train.csv_.

### Load Kaggle training data and use entire data to train tuned model

In [9]:
# Set target variable name
target = 'Demanda_uni_equil'

# Set X_train and y_train
X_train = df_train2.drop([target], axis=1)
y_train = df_train2[target]

In [10]:
# Build tuned model
#create linear regression object
#lm = linear_model.LinearRegression()

#create random forest object
rf = ensemble.RandomForestRegressor(n_estimators=50)
#n_estimators is too memory instensive for 30gb of ram, so trying 50

#train the model using the training data
#lm.fit(X_train,y_train)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [11]:

# Score tuned model
print "Score: %0.3f" % rf.score(X_train, y_train)
#score is .906 when n_estimators=10, .93 when n_estimators = 50

Score: 0.930


### Load Kaggle test data, make predictions using model, and generate submission file

In [19]:
df_test.head()

,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277


In [21]:
#create data frame for submission
df_sub = df_test[['id']]

#df_test2 = df_test2.drop('id', axis=1)

#predict using tuned model
df_sub['Demanda_uni_equil'] = rf.predict(df_test2)

df_sub.describe()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,Demanda_uni_equil
count,6.999251e+06,6.999251e+06
mean,3.499625e+06,7.727879e+00
std,2.020510e+06,1.793568e+01
min,0.000000e+00,0.000000e+00
25%,1.749812e+06,2.540000e+00
50%,3.499625e+06,4.200000e+00
75%,5.249438e+06,7.600000e+00
max,6.999250e+06,3.219380e+03


In [22]:
d = df_sub['Demanda_uni_equil']
d[d<0] = 0
df_sub.describe()


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2885: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,Demanda_uni_equil
count,6.999251e+06,6.999251e+06
mean,3.499625e+06,7.727879e+00
std,2.020510e+06,1.793568e+01
min,0.000000e+00,0.000000e+00
25%,1.749812e+06,2.540000e+00
50%,3.499625e+06,4.200000e+00
75%,5.249438e+06,7.600000e+00
max,6.999250e+06,3.219380e+03


In [23]:
# Write submission file
df_sub.to_csv("mysubmission5.csv", index=False)

**Kaggle score** : 
using linear regression 0.75682
using random forest: 0.65784


In [ ]:
#notes
#do somethign wtih time series to check for seasonality or general trending?
#look into boosting
#look into products features